<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# IBM Watson OpenScale and Batch Processing:<br>Remote Spark

## Contents

* [1. Setup](#setup)
* [2. Configure Watson OpenScale](#openscale)
* [3. Set up a subscription](#subscription)
* [4. Quality monitoring](#quality)
* [5. Drift monitoring](#drift)
* [6. Drift v2 monitoring](#drift_v2)
* [7. Fairness monitoring](#fairness)
* [8. Explainability monitoring](#explainability)

# 1. Setup <a name="setup"></a>

## Package installation

First import some of the packages you need to use. After you finish installing the following software packages, restart the kernel.



In [81]:
import warnings
warnings.filterwarnings('ignore')
%env PIP_DISABLE_PIP_VERSION_CHECK=1

env: PIP_DISABLE_PIP_VERSION_CHECK=1


In [ ]:
!pip install -U ibm-watson-openscale --no-cache | tail -n 1

In [83]:
!pip show ibm-watson-openscale

Name: ibm-watson-openscale
Version: 3.0.42.2
Summary: Client library for IBM Watson OpenScale
Home-page: https://github.ibm.com/watson-developer-cloud/openscale-python-sdk
Author: IBM Watson OpenScale
Author-email: kishore.patel@in.ibm.com
License: Apache 2.0
Location: /opt/conda/envs/Python-RT24.1-Premium/lib/python3.11/site-packages
Requires: ibm-cloud-sdk-core, pandas, python-dateutil, requests
Required-by: 


## Configure credentials

Provide your IBM Watson OpenScale credentials in the following cell:



In [84]:
WOS_CREDENTIALS = {
    "url": "", 
    "instance_id": "",
    "version": "",
    "username": "",
    "password": ""
}

## Specify model details

### Serviceprovider and subscription metadata

In [85]:
# Service Provider

SERVICE_PROVIDER_NAME = ""
SERVICE_PROVIDER_DESCRIPTION = ""

# Subscription

SUBSCRIPTION_NAME = ""
SUBSCRIPTION_DESCRIPTION = ""

### Spark Cluster

Make sure that the Apache Spark manager on the Spark cluster is running, and then provide the following details:

- SPARK_ENGINE_ENDPOINT: _Endpoint URL where the Spark Manager Application is running_
- SPARK_ENGINE_USERNAME: _Username to connect to Spark Manager Application_
- SPARK_ENGINE_PASSWORD: _Password to connect to Spark Manager Application_
- SPARK_ENGINE_NAME: _Custom display name for the Spark Manager Application_
- SPARK_ENGINE_DESCRIPTION: _Custom description for the Spark Manager Application_

In [86]:
SPARK_ENGINE_NAME=""
SPARK_ENGINE_DESCRIPTION=""
SPARK_ENGINE_ENDPOINT=""
SPARK_ENGINE_ENDPOINT_USERNAME=""
SPARK_ENGINE_ENDPOINT_PASSWORD=""

#### Provide Spark Resource Settings

To configure how much of your Spark Cluster resources this job can consume, edit the following values:


- max_num_executors: _Maximum Number of executors to launch for this session_
- min_executors: _Minimum Number of executors to launch for this session_
- executor_cores: _Number of cores to use for each executor_  
- executor_memory: _Amount of memory (in GBs) to use per executor process_
- driver_cores: _Number of cores to use for the driver process_
- driver_memory: _Amount of memory (in GBs) to use for the driver process_

In [87]:
spark_parameters = {
    "max_num_executors": 2,
    "min_num_executors": 1,
    "executor_cores": 3,
    "executor_memory": 2,
    "driver_cores": 2,
    "driver_memory": 2
}

### Apache Hive

To connect to Apache Hive, you must provide the following details:

- HIVE_CONNECTION_NAME: _Custom display name for the Hive Connection_
- HIVE_CONNECTION_DESCRIPTION: _Custom description for the Hive connection_
- [Optional] HIVE_METASTORE_URI: _Thrift URI for Hive Metastore to connect to_<br>If the metastore URI is already configured in the `hive-site.xml` file in your Hadoop Ecosystem, you can leave the `HIVE_METASTORE_URI` as `None`.

In [88]:
HIVE_CONNECTION_NAME = ""
HIVE_CONNECTION_DESCRIPTION = ""

# [optional]
HIVE_METASTORE_URI = None

### Feedback table metadata

The quality monitor stores metadata in the feedback table. To configure the quality monitor, you must provide the following details. To skip quality monitoring, run the following cell to initialize variables with the value of `None`.

- FEEDBACK_DATABASE_NAME: _Database name where feedback table is present_
- FEEDBACK_SCHEMA_NAME: _Schema name where feedback table is present_
- FEEDBACK_TABLE_NAME: _Name of the feedback table_

In [89]:
#feedback

FEEDBACK_DATABASE_NAME = None
FEEDBACK_TABLE_NAME = None

### Payload and drift table metadata

The drift monitor stores metadata in the payload and drift tables. To configure the drift monitor, you must provide the following details. To skip drift monitoring, run the following cell to initialize variables with the value of `None`.

- PAYLOAD_DATABASE_NAME: _Database name where payload logging table is present_
- PAYLOAD_SCHEMA_NAME: _Schema name where payload logging table is present_
- PAYLOAD_TABLE_NAME: _Name of the payload logging table_
- DRIFT_DATABASE_NAME: _Database name where drifted transactions table is present_
- DRIFT_SCHEMA_NAME: _Schema name where drifted transactions table is present_
- DRIFT_TABLE_NAME: _Name of the drifted transactions table_


In [90]:
#payload logging

PAYLOAD_DATABASE_NAME = None
PAYLOAD_TABLE_NAME = None

#drift

DRIFT_DATABASE_NAME = None
DRIFT_TABLE_NAME = None

### Explainability table metadata

The explainability monitor requires the queue and result tables. The payload table can also be used as the queue table. To configure the explainability monitor, you must provide the following details. To skip explainability monitoring, run the following cell to initialize variables with the value of `None`.

- EXPLAINABILITY_DATABASE_NAME: _Database name where explanations queue, result tables are present_
- EXPLAINABILITY_QUEUE_TABLE_NAME: _Name of the explanations queue table_
- EXPLAINABILITY_RESULT_TABLE_NAME: _Name of the explanations result table_

In [91]:
#explainability

EXPLAINABILITY_DATABASE_NAME = None
EXPLAINABILITY_QUEUE_TABLE_NAME = None
EXPLAINABILITY_RESULT_TABLE_NAME = None

# 2. Configure Watson OpenScale <a name="openscale"></a>

### Import the required libraries and set up the Watson OpenScale client

In [92]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import *

authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS["url"],
        username=WOS_CREDENTIALS["username"],
        password=WOS_CREDENTIALS["password"],
        disable_ssl_verification=True
    )

wos_client = APIClient(authenticator=authenticator, service_url=WOS_CREDENTIALS["url"], service_instance_id=WOS_CREDENTIALS["instance_id"])

### Display Watson OpenScale datamart details

In [93]:
wos_client.data_marts.show()
data_marts = wos_client.data_marts.list().result.data_marts
data_mart_id=data_marts[0].metadata.id

### Create a service provider

In [94]:
# Delete existing service provider with the same name as provided

service_providers = wos_client.service_providers.list().result.service_providers
for provider in service_providers:
    if provider.entity.name == SERVICE_PROVIDER_NAME:
        wos_client.service_providers.delete(service_provider_id=provider.metadata.id)
        break

In [95]:
# Add Service Provider

added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.CUSTOM_MACHINE_LEARNING,
        credentials={},
        operational_space_id="production",
        background_mode=False
    ).result

service_provider_id = added_service_provider_result.metadata.id

wos_client.service_providers.show()




 Waiting for end of adding service provider 0b5e2615-43a8-47d0-9a5e-dac79c2b229f 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




Note: First 10 records were displayed.


In [ ]:
service_provide_details = wos_client.service_providers.get(service_provider_id=service_provider_id).result
print(service_provide_details)

### Create integrated systems for Spark Engine and Hive

In [97]:
# Delete existing spark and hive integrated systems if present

integrated_systems = IntegratedSystems(wos_client).list().result.integrated_systems

for system in integrated_systems:
    if system.entity.name in (SPARK_ENGINE_NAME, HIVE_CONNECTION_NAME):
        print("Deleting integrated system {}".format(system.entity.name))
        IntegratedSystems(wos_client).delete(integrated_system_id=system.metadata.id)

Deleting integrated system GCR-HIVE-REMOTE
Deleting integrated system IAEBatchSpark


#### Spark Engine

In [ ]:
spark_engine_details = IntegratedSystems(wos_client).add(
    name=SPARK_ENGINE_NAME,
    description=SPARK_ENGINE_DESCRIPTION,
    type="spark",
    credentials={
        "username": SPARK_ENGINE_ENDPOINT_USERNAME,
        "password": SPARK_ENGINE_ENDPOINT_PASSWORD
    },
    connection={
        "endpoint": SPARK_ENGINE_ENDPOINT,
        "location_type": "custom"
    }
).result

spark_engine_id = spark_engine_details.metadata.id
print(spark_engine_details)

#### Hive

In [ ]:
hive_connection = {}
if HIVE_METASTORE_URI is not None:
    hive_connection["metastore_url"] = HIVE_METASTORE_URI
    hive_connection["location_type"] = "metastore"

hive_connection_details = IntegratedSystems(wos_client).add(
    name=HIVE_CONNECTION_NAME,
    description=HIVE_CONNECTION_DESCRIPTION,
    type="hive",
    credentials={},
    connection=hive_connection
).result

hive_connection_id=hive_connection_details.metadata.id
print(hive_connection_details)

# 3. Set up a subscription <a name="subscription"></a>

In [100]:
# Delete an existing subscription with the provided name

subscriptions = wos_client.subscriptions.list().result.subscriptions
for sub in subscriptions:
    if sub.entity.deployment.name == SUBSCRIPTION_NAME:
        wos_client.subscriptions.delete(subscription_id=sub.metadata.id)
        break

# Display all subscriptions
wos_client.subscriptions.show()

Note: First 10 records were displayed.


### Set subscription metadata

In the following cell, type a path to the common configuration JSON file that you created by running the [common configuration notebook](https://github.com/IBM/watson-openscale-samples/blob/main/Cloud%20Pak%20for%20Data/Batch%20Support/Configuration%20generation%20for%20OpenScale%20batch%20subscription.ipynb). After you edit the path information, run the cell to set the asset details and properties, the deployment details, the analytics engine details, and to add the required tables as data sources.

In [101]:
import uuid
import json

common_configuration = None
with open("/path/to/dir/containing/configuration.json", "r") as fp:
    configuration_json = json.load(fp)
    common_configuration = configuration_json.get("common_configuration")
    if common_configuration is None:
        print("Please provide the correct path to the common configuration JSON")
    
# Set asset details
asset = Asset(
    asset_id=str(uuid.uuid4()),
    url="",
    name=SUBSCRIPTION_NAME,
    asset_type=AssetTypes.MODEL,
    input_data_type=InputDataType.STRUCTURED,
    problem_type=ProblemType.BINARY_CLASSIFICATION
)

# Set deployment details
asset_deployment = AssetDeploymentRequest(
    deployment_id=str(uuid.uuid4()),
    name=SUBSCRIPTION_NAME,
    description=SUBSCRIPTION_DESCRIPTION,
    deployment_type="batch"
)

# Set asset properties 
asset_properties_request = AssetPropertiesRequest(
    label_column=common_configuration["label_column"],
    probability_fields=[common_configuration["probability"]],
    prediction_field=common_configuration["prediction"],
    feature_fields=common_configuration["feature_columns"],
    categorical_fields=common_configuration["categorical_columns"],
    training_data_schema=SparkStruct.from_dict(common_configuration["training_data_schema"]),
    input_data_schema=SparkStruct.from_dict(common_configuration["input_data_schema"]),
    output_data_schema=SparkStruct.from_dict(common_configuration["output_data_schema"])
)

# Set analytics engine details
analytics_engine = AnalyticsEngine(
    type="spark",
    integrated_system_id=spark_engine_id,
    parameters = spark_parameters
)

# Add selected tables as data sources
data_sources = []
if FEEDBACK_DATABASE_NAME is not None and FEEDBACK_TABLE_NAME is not None:
    feedback_data_source = DataSource(
        type="feedback", 
        database_name=FEEDBACK_DATABASE_NAME, 
        schema_name=FEEDBACK_DATABASE_NAME, 
        table_name=FEEDBACK_TABLE_NAME, 
        connection=DataSourceConnection(
            type="hive", 
            integrated_system_id=hive_connection_id
        ),
        parameters={
            "hive_storage_format": "" #supported values are "csv", "parquet", "orc"
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    data_sources.append(feedback_data_source)
    
if PAYLOAD_DATABASE_NAME is not None and PAYLOAD_TABLE_NAME is not None \
    and DRIFT_DATABASE_NAME is not None and DRIFT_TABLE_NAME is not None:
    payload_logging_data_source = DataSource(
        type="payload", 
        database_name=PAYLOAD_DATABASE_NAME, 
        schema_name=PAYLOAD_DATABASE_NAME, 
        table_name=PAYLOAD_TABLE_NAME, 
        connection=DataSourceConnection(
            type="hive", 
            integrated_system_id=hive_connection_id
        ),
        parameters={
            "hive_storage_format": "" #supported values are "csv", "parquet", "orc"
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    drifted_transactions_table_data_source = DataSource(
        type="drift", 
        database_name=DRIFT_DATABASE_NAME, 
        schema_name=DRIFT_DATABASE_NAME, 
        table_name=DRIFT_TABLE_NAME, 
        connection=DataSourceConnection(
            type="hive", 
            integrated_system_id=hive_connection_id
        ),
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(payload_logging_data_source)
    data_sources.append(drifted_transactions_table_data_source)

if EXPLAINABILITY_DATABASE_NAME is not None and \
    EXPLAINABILITY_QUEUE_TABLE_NAME is not None and \
        EXPLAINABILITY_RESULT_TABLE_NAME is not None:

    explainability_queue_data_source = DataSource(
        type="explain_queue", 
        database_name=EXPLAINABILITY_DATABASE_NAME,
        schema_name=EXPLAINABILITY_DATABASE_NAME,
        table_name=EXPLAINABILITY_QUEUE_TABLE_NAME,
        connection=DataSourceConnection(
            type="hive", 
            integrated_system_id=hive_integrated_system_id
        ),
        parameters={
            "hive_storage_format": "" #supported values are "csv", "parquet", "orc"
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(explainability_queue_data_source)

    explainability_result_data_source = DataSource(
        type="explain_result", 
        database_name=EXPLAINABILITY_DATABASE_NAME,
        schema_name=EXPLAINABILITY_DATABASE_NAME,
        table_name=EXPLAINABILITY_RESULT_TABLE_NAME,
        connection=DataSourceConnection(
            type="hive", 
            integrated_system_id=hive_integrated_system_id
        ),
        parameters={},
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(explainability_result_data_source)

[<ibm_watson_openscale.base_classes.watson_open_scale_v2.DataSource object at 0x7f9c4e219010>, <ibm_watson_openscale.base_classes.watson_open_scale_v2.DataSource object at 0x7f9c4e218410>, <ibm_watson_openscale.base_classes.watson_open_scale_v2.DataSource object at 0x7f9c4e21a810>, <ibm_watson_openscale.base_classes.watson_open_scale_v2.DataSource object at 0x7f9c4e21aa50>, <ibm_watson_openscale.base_classes.watson_open_scale_v2.DataSource object at 0x7f9c4e21ab50>]


In [102]:
# Adding the subscription

subscription_details = Subscriptions(wos_client).add(
    data_mart_id=data_mart_id,
    service_provider_id=service_provider_id,
    asset=asset,
    deployment=asset_deployment,
    asset_properties=asset_properties_request,
    analytics_engine=analytics_engine,
    data_sources=data_sources).result

subscription_id = subscription_details.metadata.id
print(subscription_details)

{
  "metadata": {
    "id": "d095e7f3-8419-4538-bf4a-87b55d4573b9",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:subscription:d095e7f3-8419-4538-bf4a-87b55d4573b9",
    "url": "/v2/subscriptions/d095e7f3-8419-4538-bf4a-87b55d4573b9",
    "created_at": "2025-02-13T21:29:32.541000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "service_provider_id": "0b5e2615-43a8-47d0-9a5e-dac79c2b229f",
    "asset": {
      "asset_id": "13551d9f-0a03-4c66-a60f-c6148afe4c0b",
      "url": "",
      "name": "Biswas-GCR-Drift-V2-Batch-Remote",
      "asset_type": "model",
      "problem_type": "binary",
      "input_data_type": "structured"
    },
    "asset_properties": {
      "training_data_schema": {
        "type": "struct",
        "fields": [
          {
            "metadata": {
              "columnInfo": {
                "columnLength": 64
              },
              "modeli

In [103]:
import time
# Checking subscription status

state = wos_client.subscriptions.get(subscription_id).result.entity.status.state
while state not in ["active", "error"]:
    state = wos_client.subscriptions.get(subscription_id).result.entity.status.state
    print(state)
    time.sleep(15)

In [105]:
# Checking subscription status
import time 
subscription_status = None
while subscription_status not in ("active", "error"):
    subscription_status = wos_client.subscriptions.get(subscription_id).result.entity.status.state
    if subscription_status not in ("active", "error"):
        print(datetime.now().strftime("%H:%M:%S"), subscription_status)
        time.sleep(15)
        
print(datetime.now().strftime("%H:%M:%S"), subscription_status)

21:30:09 preparing
21:30:24 active


# 4. Quality monitoring <a name="quality"></a>

### Enable the quality monitor

In the following code cell, default values are set for the quality monitor. You can change the default values by updating the optional `min_feedback_data_size` attribute in the `parameters` dict and set the quality threshold in the `thresholds` list.

In [106]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    "min_feedback_data_size": 10
}

thresholds = [{
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
}]

quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

quality_monitor_instance_id = quality_monitor_details.metadata.id
print(quality_monitor_details)

{
  "metadata": {
    "id": "bd9ec802-b579-48b7-a858-9809fb276b0f",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:bd9ec802-b579-48b7-a858-9809fb276b0f",
    "url": "/v2/monitor_instances/bd9ec802-b579-48b7-a858-9809fb276b0f",
    "created_at": "2025-02-13T21:30:30.859000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "quality",
    "target": {
      "target_type": "subscription",
      "target_id": "d095e7f3-8419-4538-bf4a-87b55d4573b9"
    },
    "parameters": {
      "min_feedback_data_size": 10
    },
    "thresholds": [
      {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "type": "relative",
        "delay_unit": "minute",
        "delay": 10
      },
      "rep

### Check monitor instance status

In [107]:
quality_status = None
from datetime import datetime

while quality_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=quality_monitor_instance_id).result
    quality_status = monitor_instance_details.entity.status.state
    if quality_status not in ("active", "error"):
        print(datetime.utcnow().strftime('%H:%M:%S'), quality_status)
        time.sleep(30)
        
print(datetime.utcnow().strftime('%H:%M:%S'), quality_status)

21:30:31 active


In [108]:
monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=quality_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "bd9ec802-b579-48b7-a858-9809fb276b0f",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:bd9ec802-b579-48b7-a858-9809fb276b0f",
    "url": "/v2/monitor_instances/bd9ec802-b579-48b7-a858-9809fb276b0f",
    "created_at": "2025-02-13T21:30:30.859000Z",
    "created_by": "cpadmin",
    "modified_at": "2025-02-13T21:30:31.330000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "quality",
    "target": {
      "target_type": "subscription",
      "target_id": "d095e7f3-8419-4538-bf4a-87b55d4573b9"
    },
    "parameters": {
      "min_feedback_data_size": 10
    },
    "thresholds": [
      {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "t

### Run an on-demand evaluation

In [109]:
# Check Quality monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=quality_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "bd9ec802-b579-48b7-a858-9809fb276b0f",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:bd9ec802-b579-48b7-a858-9809fb276b0f",
    "url": "/v2/monitor_instances/bd9ec802-b579-48b7-a858-9809fb276b0f",
    "created_at": "2025-02-13T21:30:30.859000Z",
    "created_by": "cpadmin",
    "modified_at": "2025-02-13T21:30:31.330000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "quality",
    "target": {
      "target_type": "subscription",
      "target_id": "d095e7f3-8419-4538-bf4a-87b55d4573b9"
    },
    "parameters": {
      "min_feedback_data_size": 10
    },
    "thresholds": [
      {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "t

In [110]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=
                                                          quality_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "45e00620-b24b-4523-bf3f-94f6476b76bd",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:45e00620-b24b-4523-bf3f-94f6476b76bd",
    "url": "/v2/monitor_instances/bd9ec802-b579-48b7-a858-9809fb276b0f/runs/45e00620-b24b-4523-bf3f-94f6476b76bd",
    "created_at": "2025-02-13T21:30:36.810000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "min_feedback_data_size": 10
    },
    "status": {
      "state": "running",
      "queued_at": "2025-02-13T21:30:36.804000Z",
      "started_at": "2025-02-13T21:30:36.810000Z",
      "operators": []
    }
  }
}


In [111]:
# Check run status
import time
quality_run_status = None
while quality_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=quality_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    quality_run_status = monitoring_run_details.entity.status.state
    if quality_run_status not in ("finished", "error"):
        print(datetime.utcnow().strftime("%H:%M:%S"), quality_run_status)
        time.sleep(30)
    else:
        print(monitoring_run_details.entity)
print(datetime.utcnow().strftime("%H:%M:%S"), quality_run_status)

21:30:37 running
21:31:07 running
21:31:38 running
21:32:08 running
21:32:38 running
21:33:08 running
21:33:38 running
21:34:08 running
21:34:38 running
21:35:08 running
21:35:38 running
{
  "triggered_by": "user",
  "parameters": {
    "job_run_schedule_id": "9f55db51-486c-4839-85de-805010194b64",
    "min_feedback_data_size": 10,
    "total_records_processed": 1000
  },
  "status": {
    "state": "finished",
    "queued_at": "2025-02-13T21:30:36.804000Z",
    "started_at": "2025-02-13T21:30:37.596000Z",
    "updated_at": "2025-02-13T21:35:39.709000Z",
    "completed_at": "2025-02-13T21:35:39.649000Z",
    "message": "Quality execution is completed.",
    "operators": [
      {
        "id": "original",
        "status": {
          "state": "finished",
          "completed_at": "2025-02-13T21:35:39.649000Z"
        }
      }
    ]
  }
}
21:36:08 finished


### Display quality metrics

In [112]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

# 5. Drift monitoring <a name="drift"></a>

### Enable the drift monitor

In the following code cell, type a path to the drift configuration tar ball.

In [114]:
wos_client.monitor_instances.upload_drift_model(
    model_path="/path/to/dir/containing/drift.tar.gz",
    data_mart_id=data_mart_id,
    subscription_id=subscription_id
).result

{'data_constraints': {'id': 'eb7b77e5-29ab-4ac9-a60e-92a0fbfc2acc',
  'version': '0.02_batch',
  'columns': [{'name': 'Age',
    'dtype': 'numeric_discrete',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'CheckingStatus',
    'dtype': 'categorical',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'CreditHistory',
    'dtype': 'categorical',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'CurrentResidenceDuration',
    'dtype': 'numeric_discrete',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'Dependents',
    'dtype': 'numeric_discrete',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'EmploymentDuration',
    'dtype': 'categorical',
    'count': 1000,
    'appr

In the following code cell, default values are set for the drift monitor. You can change the default values by updating the values in the parameters section. The `min_samples` parameter controls the number of records that triggers the drift monitor to run. The `drift_threshold` parameter sets the threshold in decimal format for the drift percentage to trigger an alert. The `train_drift_model` parameter controls whether to re-train the model based on the drift analysis.



In [115]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    "min_samples": 100,
    "drift_threshold": 0.05,
    "train_drift_model": False
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
print(drift_monitor_details)

{
  "metadata": {
    "id": "de2d4c5b-e0d5-47c3-8964-3f744f0e8226",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:de2d4c5b-e0d5-47c3-8964-3f744f0e8226",
    "url": "/v2/monitor_instances/de2d4c5b-e0d5-47c3-8964-3f744f0e8226",
    "created_at": "2025-02-13T21:36:10.976000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "drift",
    "target": {
      "target_type": "subscription",
      "target_id": "d095e7f3-8419-4538-bf4a-87b55d4573b9"
    },
    "parameters": {
      "drift_threshold": 0.05,
      "min_samples": 100,
      "train_drift_model": false
    },
    "thresholds": [
      {
        "metric_id": "drift_magnitude",
        "type": "upper_limit",
        "value": 0.5
      },
      {
        "metric_id": "predicted_accuracy",
        "type": "upper_limit",
        "value": 0.8
      },
      {
        "metric_id": "data_d

### Check monitor instance status

In [116]:
drift_status = None

while drift_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_monitor_instance_id).result
    drift_status = monitor_instance_details.entity.status.state
    if drift_status not in ("active", "error"):
        print(datetime.utcnow().strftime('%H:%M:%S'), drift_status)
        time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), drift_status)

21:36:11 preparing
21:36:41 preparing
21:37:11 preparing
21:37:41 preparing
21:38:11 preparing
21:38:41 active


### Run an on-demand evaluation

In [117]:
# Check Drift monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "de2d4c5b-e0d5-47c3-8964-3f744f0e8226",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:de2d4c5b-e0d5-47c3-8964-3f744f0e8226",
    "url": "/v2/monitor_instances/de2d4c5b-e0d5-47c3-8964-3f744f0e8226",
    "created_at": "2025-02-13T21:36:10.976000Z",
    "created_by": "cpadmin",
    "modified_at": "2025-02-13T21:38:15.015000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "drift",
    "target": {
      "target_type": "subscription",
      "target_id": "d095e7f3-8419-4538-bf4a-87b55d4573b9"
    },
    "parameters": {
      "config_status": {
        "model_name": null,
        "state": "finished"
      },
      "data_drift_enabled": true,
      "data_drift_threshold": 0.1,
      "drift_buffer_range": [
        -4.5,
        4.5
      ],
      "drift_model_version": null,
      "drift_threshold": 0

In [118]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "fb79e9b3-f89a-4292-bd35-28b20173f2cb",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:fb79e9b3-f89a-4292-bd35-28b20173f2cb",
    "url": "/v2/monitor_instances/de2d4c5b-e0d5-47c3-8964-3f744f0e8226/runs/fb79e9b3-f89a-4292-bd35-28b20173f2cb",
    "created_at": "2025-02-13T21:38:41.815000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "config_status": {
        "model_name": null,
        "state": "finished"
      },
      "data_drift_enabled": true,
      "data_drift_threshold": 0.1,
      "drift_buffer_range": [
        -4.5,
        4.5
      ],
      "drift_model_version": null,
      "drift_threshold": 0.05,
      "min_samples": 100,
      "model_drift_enabled": true,
      "table_schema": {
        "fields": [
          {
            "length": 64,
            "metadata": {},
            "name": "scoring_id",
            "nullable": false,
       

In [119]:
# Check run status

drift_run_status = None
while drift_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=drift_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    drift_run_status = monitoring_run_details.entity.status.state
    if drift_run_status not in ("finished", "error"):
        print(datetime.utcnow().strftime("%H:%M:%S"), drift_run_status)
        time.sleep(30)
        
print(datetime.utcnow().strftime("%H:%M:%S"), drift_run_status)

21:38:41 running
21:39:12 running
21:39:42 running
21:40:12 running
21:40:42 running
21:41:12 finished


### Display drift metrics

In [120]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

# 6. Drift V2 monitoring <a name="drift_v2"></a>

### Enable the drift v2 monitor

In the following code cell, type a path to the drift v2 configuration tar ball.

In [121]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift_v2" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift_v2 monitor instance with id: ', monitor_instance.metadata.id)

In [123]:
wos_client.monitor_instances.upload_drift_v2_archive(
    archive_path="/path/to/dir/containing/drift_v2_archive.tar.gz",
    data_mart_id=data_mart_id,
    subscription_id=subscription_id
).result

{}

In [124]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
        "min_samples": 10,
        "max_samples": 1000,
        "train_archive": False
    }
drift_v2_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT_V2.ID,
    target=target,
    parameters=parameters
).result

drift_v2_monitor_instance_id = drift_v2_monitor_details.metadata.id
drift_v2_monitor_instance_id

'198943bc-da59-45d7-ab86-4cf3cc869356'

### Check monitor instance status

In [125]:
import time 
drift_v2_status = None

while drift_v2_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_v2_monitor_instance_id).result
    drift_v2_status = monitor_instance_details.entity.status.state
    if drift_v2_status not in ("active", "error"):
        print(datetime.now().strftime("%H:%M:%S"), drift_v2_status)
        time.sleep(30)

print(datetime.now().strftime("%H:%M:%S"), drift_v2_status)

21:41:14 preparing
21:41:44 active


### Trigger on-demand monitor run

In [126]:
# Check Drift v2 monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_v2_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "198943bc-da59-45d7-ab86-4cf3cc869356",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:198943bc-da59-45d7-ab86-4cf3cc869356",
    "url": "/v2/monitor_instances/198943bc-da59-45d7-ab86-4cf3cc869356",
    "created_at": "2025-02-13T21:41:13.993000Z",
    "created_by": "cpadmin",
    "modified_at": "2025-02-13T21:41:26.563000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "drift_v2",
    "target": {
      "target_type": "subscription",
      "target_id": "d095e7f3-8419-4538-bf4a-87b55d4573b9"
    },
    "parameters": {
      "advanced_controls": {
        "enable_drift_v2_batch": true
      },
      "context": {
        "baseline": {
          "data_set_id": "e32f39c1-e782-4a55-89c6-66eb4e7bb3b1"
        },
        "copy_subscription": {
          "enabled": false,
          "source_id": null
   

In [127]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_v2_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "bf8affc1-8cc4-4615-9ab9-41dd9e54ffa6",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:bf8affc1-8cc4-4615-9ab9-41dd9e54ffa6",
    "url": "/v2/monitor_instances/198943bc-da59-45d7-ab86-4cf3cc869356/runs/bf8affc1-8cc4-4615-9ab9-41dd9e54ffa6",
    "created_at": "2025-02-13T21:41:44.427000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "advanced_controls": {
        "enable_drift_v2_batch": true
      },
      "context": {
        "baseline": {
          "data_set_id": "e32f39c1-e782-4a55-89c6-66eb4e7bb3b1"
        },
        "copy_subscription": {
          "enabled": false,
          "source_id": null
        },
        "data_sets": {
          "drift_insights": "1ed83330-9a53-4d26-9119-13aec67b4588",
          "drift_intervals": "032afd3b-b548-4ce7-8cc7-003ff5cb6870",
          "drift_scores": "2c823534-b900-4a6f-b0f4-a90309b8b536",
          "drift_

In [128]:
# Check run status
drift_v2_run_status = None
while drift_v2_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=drift_v2_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    drift_v2_run_status = monitoring_run_details.entity.status.state
    if drift_v2_run_status not in ("finished", "error"):
        print(datetime.now().strftime("%H:%M:%S"), drift_v2_run_status)
        time.sleep(30)
        
print(datetime.now().strftime("%H:%M:%S"), drift_v2_run_status)

21:41:44 running
21:42:14 running
21:42:44 running
21:43:14 running
21:43:44 running
21:44:15 running
21:44:45 running
21:45:15 running
21:45:45 running
21:46:15 running
21:46:45 running
21:47:15 running
21:47:45 running
21:48:15 running
21:48:45 running
21:49:15 running
21:49:45 running
21:50:15 running
21:50:46 running
21:51:16 running
21:51:46 running
21:52:16 running
21:52:46 running
21:53:16 running
21:53:46 running
21:54:16 finished


### Check Drift V2 metrics

In [129]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_v2_monitor_instance_id)

Note: First 10 records were displayed.


# 6. Fairness monitoring <a name="fairness"></a>

### Enable the fairness monitor

The following code cell, will enable the fairness monitor.

In [130]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = configuration_json["fairness_configuration"]["parameters"]
thresholds = configuration_json["fairness_configuration"]["thresholds"]

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

fairness_monitor_instance_id = fairness_monitor_details.metadata.id
print(fairness_monitor_details)

{
  "metadata": {
    "id": "7e81a650-0a32-4852-af23-8393e7dbebcd",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:7e81a650-0a32-4852-af23-8393e7dbebcd",
    "url": "/v2/monitor_instances/7e81a650-0a32-4852-af23-8393e7dbebcd",
    "created_at": "2025-02-13T21:54:18.876000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "fairness",
    "target": {
      "target_type": "subscription",
      "target_id": "d095e7f3-8419-4538-bf4a-87b55d4573b9"
    },
    "parameters": {
      "training_data_last_processed_time": "2025-01-17T09:40:55.219953Z",
      "min_records": 100,
      "features": [
        {
          "minority": [
            "female"
          ],
          "feature": "Sex",
          "metric_ids": [
            "fairness_value",
            "statistical_parity_difference",
            "average_odds_difference",
            "av

### Check monitor instance status

In [131]:
fairness_state = fairness_monitor_details.entity.status.state

while fairness_state not in ("active", "error"):
    print(datetime.utcnow().strftime('%H:%M:%S'), fairness_state)
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=fairness_monitor_instance_id).result
    fairness_state = monitor_instance_details.entity.status.state
    time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), fairness_state)

21:54:18 preparing
21:54:49 preparing
21:55:19 active


### Run an on-demand evaluation

In [132]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "1bce40e5-ce94-4589-8d1e-66c9c0017062",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:1bce40e5-ce94-4589-8d1e-66c9c0017062",
    "url": "/v2/monitor_instances/7e81a650-0a32-4852-af23-8393e7dbebcd/runs/1bce40e5-ce94-4589-8d1e-66c9c0017062",
    "created_at": "2025-02-13T21:55:19.299000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "favourable_class": [
        "No Risk"
      ],
      "features": [
        {
          "feature": "Sex",
          "majority": [
            "male"
          ],
          "metric_ids": [
            "fairness_value",
            "statistical_parity_difference",
            "average_odds_difference",
            "average_abs_odds_difference",
            "false_negative_rate_difference",
            "false_positive_rate_difference",
            "false_discovery_rate_difference",
            "false_omission_rate_differenc

In [133]:
# Check run status

fairness_run_status = monitoring_run_details.entity.status.state
while fairness_run_status not in ("finished", "error"):
    print(datetime.utcnow().strftime("%H:%M:%S"), fairness_run_status)
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=fairness_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    fairness_run_status = monitoring_run_details.entity.status.state
    time.sleep(30)
        
print(datetime.utcnow().strftime("%H:%M:%S"), fairness_run_status)

21:55:19 running
21:55:49 running
21:56:19 running
21:56:49 running
21:57:19 running
21:57:49 running
21:58:19 running
21:58:50 running
21:59:20 running
21:59:50 running
22:00:20 running
22:00:50 running
22:01:21 finished


### Display fairness metrics

In [134]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

Note: First 10 records were displayed.


# 7. Explainability monitoring <a name="explainability"></a>

### Enable the explainability monitor

#### Upload explainability configuration archive
In the following code cell, type the path to the explainability configuration archive tar ball.

In [168]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "explainability" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing explainability monitor instance with id: ', monitor_instance.metadata.id)

Deleted existing explainability monitor instance with id:  babd23f2-151d-4de6-89d2-651b536280e6


In [170]:
with open("/path/to/dir/containing/explainability.tar.gz", mode="rb") as explainability_tar:
    wos_client.monitor_instances.upload_explainability_archive(subscription_id=subscription_id, archive=explainability_tar)

print("Uploaded explainability archive successfully.")

Uploaded explainability archive successfully.


In [173]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
# Uncomment the below lines to enable lime global explanation. Available from Cloud Pak for Data version 4.6.4 onwards.
   # "global_explanation": {
   #     "enabled": True,  # Flag to enable global explanation 
   #     "explanation_method": "lime",
       # "sample_size": 1000, # [Optional] The sample size of records to be used for generating payload data global explanation. If not specified entire data in the payload window is used.
   # }
}

explainability_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_instance_id = explainability_monitor_details.metadata.id
print(explainability_monitor_details)

{
  "metadata": {
    "id": "cde6daac-4235-48d0-832b-2ddce7bb5ccb",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:cde6daac-4235-48d0-832b-2ddce7bb5ccb",
    "url": "/v2/monitor_instances/cde6daac-4235-48d0-832b-2ddce7bb5ccb",
    "created_at": "2025-02-13T22:34:49.665000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "explainability",
    "target": {
      "target_type": "subscription",
      "target_id": "d095e7f3-8419-4538-bf4a-87b55d4573b9"
    },
    "parameters": {},
    "thresholds": [
      {
        "metric_id": "global_explanation_stability",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "type": "relative",
        "delay_unit": "minute",
        "delay": 10
      },
      "repeat_type": "week"
 

### Check monitor instance status

In [174]:
explainability_status = None

while explainability_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=explainability_monitor_instance_id).result
    explainability_status = monitor_instance_details.entity.status.state
    if explainability_status not in ("active", "error"):
        print(datetime.utcnow().strftime('%H:%M:%S'), explainability_status)
        time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), explainability_status)

22:34:49 preparing
22:35:19 active


### Run an on-demand evaluation

In [175]:
# Check Explainbility monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=explainability_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "cde6daac-4235-48d0-832b-2ddce7bb5ccb",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:cde6daac-4235-48d0-832b-2ddce7bb5ccb",
    "url": "/v2/monitor_instances/cde6daac-4235-48d0-832b-2ddce7bb5ccb",
    "created_at": "2025-02-13T22:34:49.665000Z",
    "created_by": "cpadmin",
    "modified_at": "2025-02-13T22:34:51.952000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "explainability",
    "target": {
      "target_type": "subscription",
      "target_id": "d095e7f3-8419-4538-bf4a-87b55d4573b9"
    },
    "parameters": {
      "config_modified_at": "2025-02-13T22:34:51.810221Z",
      "explanations_count": {
        "failed": 0,
        "total": 0
      },
      "lime": {
        "enabled": true,
        "features_count": 10,
        "perturbations_count": 10000
      }
    },
    "thresholds

In [176]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=explainability_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "10f1bba8-e26a-4b19-afd4-ff8572a7f535",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:10f1bba8-e26a-4b19-afd4-ff8572a7f535",
    "url": "/v2/monitor_instances/cde6daac-4235-48d0-832b-2ddce7bb5ccb/runs/10f1bba8-e26a-4b19-afd4-ff8572a7f535",
    "created_at": "2025-02-13T22:35:20.131000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "config_modified_at": "2025-02-13T22:34:51.810221Z",
      "explanations_count": {
        "failed": 0,
        "total": 0
      },
      "lime": {
        "enabled": true,
        "features_count": 10,
        "perturbations_count": 10000
      }
    },
    "status": {
      "state": "running",
      "queued_at": "2025-02-13T22:35:20.125000Z",
      "started_at": "2025-02-13T22:35:20.131000Z",
      "operators": []
    }
  }
}


In [177]:
# Check run status

explainability_run_status = None
while explainability_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=explainability_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    explainability_run_status = monitoring_run_details.entity.status.state
    if explainability_run_status not in ("finished", "error"):
        print(datetime.utcnow().strftime("%H:%M:%S"), explainability_run_status)
        time.sleep(60)
        
print(datetime.utcnow().strftime("%H:%M:%S"), explainability_run_status)

22:35:21 running
22:36:21 running
22:37:21 running
22:38:21 running
22:39:21 running
22:40:21 running
22:41:21 finished


In [ ]:
# View the global explanation stability metric. When lime global explanation is enabled, the monitor run computes global explanation and publishes global_explanation_stability metric.
# wos_client.monitor_instances.show_metrics(monitor_instance_id=explainability_monitor_instance_id)

### Display sample explanations

In [178]:
explanations = wos_client.monitor_instances.get_all_explaination_tasks(subscription_id=subscription_id).result
print(explanations)

{
  "total_count": 30,
  "limit": 50,
  "offset": 0,
  "explanation_fields": [
    "explanation_task_id",
    "scoring_id",
    "created_at",
    "finished_at",
    "status",
    "prediction",
    "subscription_id",
    "deployment_id",
    "asset_name",
    "deployment_name",
    "probability",
    "explanation_type"
  ],
  "explanation_values": [
    [
      "9fd6a836-29a1-4091-a064-f63b6f94cc0e",
      "f6ec8588-97da-4224-878c-7de35f499e7e",
      "2025-02-13T22:36:09.290272Z",
      "2025-02-13T22:36:09.290747Z",
      "finished",
      "No Risk",
      "d095e7f3-8419-4538-bf4a-87b55d4573b9",
      "590b2671-1f6a-4c56-b140-55874afc6a24",
      "Biswas-GCR-Drift-V2-Batch-Remote",
      "Biswas-GCR-Drift-V2-Batch-Remote",
      0.59,
      "lime"
    ],
    [
      "a0582fbf-b1cd-4222-81d2-05a128b9df04",
      "9c7f1f7c-caa3-4d07-9af4-08add8f58e43",
      "2025-02-13T22:36:09.309272Z",
      "2025-02-13T22:36:09.309449Z",
      "finished",
      "No Risk",
      "d095e7f3-8419-4538-b

## Congratulations!

You have finished the Batch demo for IBM Watson OpenScale using Remote Apache Spark. You can now view the [Watson OpenScale Dashboard](https://url-to-your-cp4d-cluster/aiopenscale). Click the tile for the **German Credit model** to see quality, drift and fairness monitors. Click the timeseries graph to get detailed information on transactions during a specific time window.